# Generating counterfactuals for multi-class classification and regression models
This notebook will demonstrate how the DiCE library can be used for multiclass classification and regression for scikit-learn models.
You can use any method ("random", "kdtree", "genetic"), just specific it in the method argument in the initialization step. The rest of the code is completely identical.
For demonstration, we will be using the genetic algorithm for CFs.

In [41]:
import dice_ml
from dice_ml import Dice

from sklearn.datasets import load_iris, fetch_california_housing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

import pandas 

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


We will use sklearn's internal datasets to demonstrate DiCE's features in this notebook

## Multiclass Classification

For multiclass classification, we will use sklearn's Iris dataset. This data set consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset

In [53]:
path = "healthcare-dataset-stroke-data.csv"

df_iris = pandas.read_csv(path)
#simplified_dataset = df_iris.loc[:,['age', 'hypertension', 'heart_disease', 'avg_glucose_level',
#                             'bmi', 'smoking_status', 'stroke']]

df_iris.dropna(inplace=True)


In [54]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4909 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 4909 non-null   int64  
 1   gender             4909 non-null   object 
 2   age                4909 non-null   float64
 3   hypertension       4909 non-null   int64  
 4   heart_disease      4909 non-null   int64  
 5   ever_married       4909 non-null   object 
 6   work_type          4909 non-null   object 
 7   Residence_type     4909 non-null   object 
 8   avg_glucose_level  4909 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     4909 non-null   object 
 11  stroke             4909 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 498.6+ KB


In [55]:
outcome_name = "stroke"
continuous_features_iris = ['age', 'avg_glucose_level', 'bmi']
target = df_iris[outcome_name]

In [56]:
# Split data into train and test
datasetX = df_iris.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=target)

categorical_features = x_train.columns.difference(continuous_features_iris)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_iris),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf_iris = Pipeline(steps=[('preprocessor', transformations),
                           ('classifier', RandomForestClassifier())])

model_iris = clf_iris.fit(x_train, y_train)

In [57]:
d_iris = dice_ml.Data(dataframe=df_iris,
                      continuous_features=continuous_features_iris,
                      outcome_name='stroke')

# We provide the type of model as a parameter (model_type)
m_iris = dice_ml.Model(model=model_iris, backend="sklearn", model_type='classifier')

In [60]:
exp_genetic_iris = Dice(d_iris, m_iris, method="kdtree")

As we can see below, all the target values will lie in the desired class

In [ ]:
# Single input
query_instances_iris = datasetX[2:3]

genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=1, desired_class=0)
genetic_iris.visualize_as_dataframe()

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is u

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

...

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  query_instance_df_dummies[col] = 0
C:\Users\fwanguem\Anaconda3\lib\site-packages\dice_ml\explainer_interfaces\dice_KD.py:228: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

In [ ]:
# Multiple queries can be given as input at once
query_instances_iris = x_test[17:19]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=7, desired_class=2)
genetic_iris.visualize_as_dataframe(show_only_changes=True)

# Regression

For regression, we will use sklearn's California Housing dataset. This dataset contains California house prices. More information at https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html

In [ ]:
housing_data = fetch_california_housing()
df_housing = pd.DataFrame(housing_data.data, columns=housing_data.feature_names)
df_housing[outcome_name] = pd.Series(housing_data.target)
df_housing.head()

In [ ]:
df_housing.info()

In [ ]:
continuous_features_housing = df_housing.drop(outcome_name, axis=1).columns.tolist()
target = df_housing[outcome_name]

In [1]:
# Split data into train and test
datasetX = df_housing.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX,
                                                    target,
                                                    test_size=0.2,
                                                    random_state=0)

categorical_features = x_train.columns.difference(continuous_features_housing)

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_housing),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
regr_housing = Pipeline(steps=[('preprocessor', transformations),
                               ('regressor', RandomForestRegressor())])
model_housing = regr_housing.fit(x_train, y_train)

NameError: name 'df_housing' is not defined

In [ ]:
d_housing = dice_ml.Data(dataframe=df_housing, continuous_features=continuous_features_housing, outcome_name=outcome_name)
# We provide the type of model as a parameter (model_type)
m_housing = dice_ml.Model(model=model_housing, backend="sklearn", model_type='regressor')

In [ ]:
exp_genetic_housing = Dice(d_housing, m_housing, method="genetic")

As we can see below, all the target values will lie in the desired range

In [ ]:
# Multiple queries can be given as input at once
query_instances_housing = x_test[2:4]
genetic_housing = exp_genetic_housing.generate_counterfactuals(query_instances_housing,
                                                               total_CFs=2,
                                                               desired_range=[3.0, 5.0])
genetic_housing.visualize_as_dataframe(show_only_changes=True)